<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

# 3.4 Pandas数值运算方法

NumPy的基本能力之一是快速对每个元素进行运算，既包括基本算术运算（加、减、乘、除），也包括更复杂的运算（三角函数、指数函数和对数函数等）。 Pandas继承了NumPy的功能，在2.3节介绍过的通用函数是关键。

但是Pandas也实现了一些高效技巧：对于一元运算（像函数与三角函数），这些通用函数将在输出结果中**保留索引和列标签**；而对于二元运算（如加法和乘），Pandas在传递通用函数时会**自动对齐索引**进行计算。这就意味着，保存数据内容与组合不同来源的数据——两处在NumPy数组中都容易出错的地方——变成了Pandas的杀手锏。后面还会介绍一些关于一维``Series``和二维``DataFrame``的便捷运算方法。

## 3.4.1通用函数： 保留索引

因为Pandas是建立在NumPy基础之上的，所以NumPy的通用函数同样适用于Pandas的``Series``和``DataFrame``对象。让我们用一个简单的``Series``和``DataFrame``来演示：

In [37]:
import pandas as pd
import numpy as np

In [38]:
rng = np.random.RandomState(42)#随机数生成器的种子
ser = pd.Series(rng.randint(0, 10, size=4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [39]:
df = pd.DataFrame(rng.randint(0, 10, size=(3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


如果对这两个对象的其中一个使用NumPy通用函数，生成的结果是**另一个保留索引**的Pandas对象：

In [40]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

或者，再做一个比较复杂的运算：

In [41]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


**任何**一种在2.3节介绍过的通用函数都可以按照类似的方式使用。

## 3.4.2 通用函数： 索引对齐

当在两个``Series``或``DataFrame``对象上进行二元计算时，Pandas会在计算过程中**对齐**两个对象的**索引**。当你处理**不完整的数据**时，这一点非常方便，我们将在后面的示例中看到。

### 1. Series索引对齐

假如你要整合两个数据源的数据：

In [42]:
population = pd.Series({'重庆': 3101.79,'上海': 2423.78,'北京': 2154.2,'成都': 1633}, name='人口')
GDP = pd.Series({'上海': 32679.87, '北京': 30320.00,'成都': 15342.77, '天津': 188099.64}, name='GDP')

来看看如果用人口除以面积会得到什么样的结果：

In [43]:
GDP / population

上海    13.483018
北京    14.074831
天津          NaN
成都     9.395450
重庆          NaN
dtype: float64

结果数组的索引是两个输入数组索引的**并集**。我们也可以用Python标准库的集合运算法则来获得这个索引：

In [44]:
population.index | GDP.index
# 在python中，字符“|”表示“管道”。希望匹配许多表达式中的一个时，就可以使用它，可以理解为“或”的意思。

Index(['上海', '北京', '天津', '成都', '重庆'], dtype='object')

对于缺失位置的数据，Pandas 会用``NaN``填充，表示“Not a Number”。这是 Pandas 表示缺失值的方法（详见下节）。

这种索引对齐方式是通过Python内置的集合运算规则实现的，任何缺失值默认都用``NaN``填充。

In [45]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

如果用NaN值不是我们想要的结果，那么可以用适当的对象方法代替运算符。例如，``A.add(B)``等价于``A + B``，也可以设置参数自定义``A``或``B``缺失的数据：

In [46]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### DataFrame索引对齐

在计算两个``DataFrame``时，类似的索引对齐规则也同样会出现在**共同**（并集）列中：

In [47]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [48]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [49]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


你会发现，两个对象的行列索引可以是不同顺序的，结果的索引会**自动按顺序排列**。在``Series``中，我们可以通过运算符方法的``fill_value``参数自定义缺失值。这里，我们将用``A``中所有值的均值来填充缺失值（计算``A``的均值需要用stack将二维数组压缩成一维数组）：

In [51]:
fill = A.stack().mean() #计算A中所有元素的均值（你可以在这里输出A.stack()看看其形式）
#print(A.stack())
#print(A.stack().mean())
A.add(B, fill_value=fill)

4.5


,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


下表列举了与 Python 运算符相对应的Pandas对象方法。

| Python运算符    | Pandas方法                            |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## 3.4.3通用函数： DataFrame与Series的运算

我们经常需要对一个``DataFrame``和一个``Series``进行计算，行列对齐方式与之前类似。也就是说，``DataFrame``和``Series``的运算规则，与NumPy中二维数组与一维数组的运算规则是一样的。来看一个常见运算，让一个二维数组减去自身的一行数据：

In [15]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [16]:
A-A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

根据NumPy的广播规则，让二维数组减自身的一行数据会按行计算。

在Pandas里默认也是按行运算的：

In [17]:
df = pd.DataFrame(A, columns=list('QRST'))
df-df.iloc[0] #iloc是根据行号来索引，行号从0开始，逐次加1

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


如果你想按列计算，那么就需要利用前面介绍过的运算符方法，通过``axis``参数设置：

In [18]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


你会发现``DataFrame``/``Series``的运算与前面介绍的运算一样，结果的索引都会自动对齐：

In [19]:
halfrow = df.iloc[0, ::2] # ::2表示隔一列取一个数字
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [20]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


这些行列索引的保留与对齐方法说明Pandas在运算时会一直保存这些数据内容，从而避免在处理数据类型有差异和/或维度不一致的NumPy数组时可能遇到的问题。

<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >